# 한번에

In [ ]:
import openai
import os
import json
import time
import logging
from dotenv import load_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
import pymysql
import locale
from langchain.prompts import PromptTemplate

# 로그 설정
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# .env 파일 로드
load_dotenv()

# OpenAI API 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ 1️⃣ 질문 분류 GPT
def classify_question(question):
    """질문을 분석하여 카테고리를 반환하는 GPT"""
    logging.info(f"🟡 [질문 분류] 입력 질문: {question}")

    system_prompt = """
    당신은 사용자의 질문을 적절한 카테고리로 분류하는 AI입니다.
    
    가능한 카테고리:
    1. 채용 정보
    2. 일반 질문
    
    사용자의 질문에 '요청'이 포함되지 않았을시 일반 질문으로 분류하세요.
    질문을 분석하여 가장 적절한 카테고리를 JSON 형식으로 출력하세요.

    **출력 형식 (JSON)**
    {"category": "채용 정보"}
    """

    ####사람들의 로그데이터 분석을 통해서 카테고리를 세분화 하겠다~. 배타적이여야함.

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ],
        temperature=0.3,
        max_tokens=50
    )

    result = json.loads(response["choices"][0]["message"]["content"])
    logging.info(f"🟢 [질문 분류 완료] 카테고리: {result['category']}")
    return result  # JSON 변환

# ✅ 2️⃣ 데이터 검색 GPT
def connect_to_db():
    """MySQL 데이터베이스 연결"""
    return pymysql.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        passwd='seul0899',
        db='saramin',
        charset="utf8mb4"
    )

def fetch_from_saramin_refined():
    """MySQL에서 채용 정보를 가져옴"""
    connection = connect_to_db()
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    cursor.execute("SELECT * FROM saramin_5_combined_revised_fordb")
    fetch_result = cursor.fetchall()
    connection.close()
    
    logging.info(f"🟢 [DB 데이터 로드 완료] 총 {len(fetch_result)}개 문서 가져옴")
    return fetch_result

def chunk_document(doc_text, chunk_size=1000, chunk_overlap=100):
    """문서를 작은 단위로 분할"""
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return text_splitter.split_text(doc_text)

def convert_to_documents(courses):
    """MySQL 데이터를 LangChain 문서로 변환"""
    documents = []
    for course in courses:
        doc_text = "\n".join([f"{key}: {value}" for key, value in course.items()])
        chunks = chunk_document(doc_text)
        documents.extend([Document(page_content=chunk) for chunk in chunks])

    logging.info(f"🟢 [문서 변환 완료] 총 {len(documents)}개 문서 생성됨")
    return documents

def create_vector_store(documents):
    """벡터 스토어를 불러오거나, 없으면 새로 생성하여 저장"""
    embeddings = OpenAIEmbeddings()
    faiss_path = './db/faiss'  # 벡터 스토어 경로

    # ✅ 벡터 스토어가 존재하면 불러오기
    if os.path.exists(faiss_path):
        logging.info("🟡 [벡터 스토어 로드] 기존 벡터 스토어를 불러옵니다.")
        store = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)
    else:
        # ❌ 존재하지 않으면 새로 생성 후 저장
        logging.info("🔵 [벡터 스토어 생성] 새로운 벡터 스토어를 만듭니다.")
        store = FAISS.from_documents(documents, embeddings)
        store.save_local(faiss_path)  # 벡터 스토어 저장
        logging.info("🟢 [벡터 스토어 저장 완료] 새로운 벡터 스토어를 저장했습니다.")

    logging.info(f"🟢 [벡터 스토어 준비 완료] 문서 개수: {len(documents)}")
    return store

def create_chatbot(vector_store):
    """채용 정보 기반 챗봇 생성"""
    chat_model = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

    prompt_template = PromptTemplate(
        input_variables=["question", "context"],  # 필요한 변수 정의
        template="""
        시스템 프롬프트:
        1. 당신은 잡Job의 채용 전문가입니다.사용자의 요구에 맞는 채용 정보를 제공하세요.
        2. 채용 공고에 대한 정보를 제공할시 다음과 같은 형식으로 답변하세요. '공고제목', '공고링크', '기업명'을 반드시 포함하세요. 개수와 관련된 명시가 없을시 기본적으로 3개의 공고를 제공하세요.
        - 공고 제목: (공고제목)
        - 공고 링크: (공고링크)
        - 기업명:(기업명)
        - 태그:(태그)
        - 주요 업무: (주요 업무에 대한 설명)
        - 자격 요건: (필수 자격 요건)

        3. 관련 없는 질문에는 자연스럽게 채용 정보 질문을 유도하세요.
        4. 직급과 관련된 답변은 포함하지 마세요. 직급과 관련한 질문에는 공고링크를 제공하세요

        질문: {question}
        
        관련 문서:
        {context}
        """
    )

    return ConversationalRetrievalChain.from_llm(
        llm=chat_model,
        retriever=vector_store.as_retriever(),
        return_source_documents=True,
        verbose=True,
        combine_docs_chain_kwargs={"prompt": prompt_template}  # 이제 PromptTemplate 객체 사용
    )

def get_answer(question, retrieval_chain):
    """채용 정보를 검색하여 응답 생성"""
    result = retrieval_chain.invoke({"question": question, "chat_history": []})

    answer = result.get("answer", "")
    logging.info(f"🟢 [답변 생성 완료] 답변: {answer[:100]}...")  # 너무 길면 자름

    return answer, result.get("source_documents", [])

# ✅ 3️⃣ 질문-답변 검증 GPT
def verify_answer(question, answer):
    """질문과 답변이 논리적으로 맞는지 검증"""
    system_prompt = """
    당신은 AI 검증 시스템입니다. 질문과 답변이 논리적으로 연결되는지 판단하세요.

    - 질문과 답변이 맞으면: {"verification": "일치"}
    - 질문과 답변이 맞지 않으면: {"verification": "불일치"}
    """
    ### 불일치 하는 과정을 확인. 불일치 비율 검증. 

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini", 
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"질문: {question}\n답변: {answer}"}
        ],
        temperature=0.3,
        max_tokens=50
    )

    result = json.loads(response["choices"][0]["message"]["content"])
    logging.info(f"🟢 [답변 검증 완료] 결과: {result['verification']}")
    return result

# ✅ 대화 기록 저장을 위한 리스트
chat_history = []

def get_answer_with_similarity_check(question, vector_store, chat_history):
    """채용 정보를 검색하여 응답을 생성하며, 유사도가 0.2 이상인지 확인"""
    retriever = vector_store.as_retriever()
    
    # 유사도 검사 (최대 3개의 문서 검색)
    docs_with_scores = retriever.vectorstore.similarity_search_with_score(question, k=3)
    
    # 가장 높은 유사도를 가진 문서 찾기
    if not docs_with_scores:
        logging.warning("❌ 관련 문서를 찾지 못했습니다.")
        return "⚠ 관련된 채용 정보를 찾지 못했습니다. 좀 더 구체적으로 질문해 주세요.", []
    
    highest_score = max(score for _, score in docs_with_scores)
    logging.info(f'✅유사도 최고 점수: {highest_score}')

    # ✅ 유사도가 0.3 이상이면 문서를 기반으로 응답 생성
    if highest_score >= 0.3:
        retrieval_chain = create_chatbot(vector_store)
        result = retrieval_chain.invoke({
            "question": question, 
            "chat_history": chat_history  # ✅ 기존 대화 기록 추가
        })
        answer = result.get("answer", "")

        # ✅ 대화 기록 업데이트
        chat_history.append({"question": question, "answer": answer})
        
        return answer, result.get("source_documents", [])

    # ❌ 유사도가 낮을 경우 다시 질문을 유도
    logging.warning("❌ 유사도가 낮아 문서를 참조하지 않습니다.")
    return "⚠ 현재 질문과 관련된 채용 정보를 찾을 수 없습니다. 조금 더 상세히 질문해 주세요!", []


# ✅ 4️⃣ 전체 실행 함수 (대화 기록 활용)
def handle_request(question):
    """질문을 처리하고, 유사도 기준을 적용하여 데이터 검색"""
    
    # 1️⃣ 질문 분류 실행
    category_result = classify_question(question)
    category = category_result["category"]
    
    if category == "일반 질문":
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system",
                "content": """당신은 잡Job의 채용 전문가입니다. 
                            당신은 채용 정보에 관한 질문에만 대답할 수 있습니다. 그 외의 질문에는 한줄의 단답형으로 답하세요.
                            '채용 공고와 관련된 질문이 있으신가요?'를 마지막에 덧붙이세요"""},
                # ✅ 대화 기록 추가
                *[
                    {"role": "user", "content": entry["question"]}
                    for entry in chat_history
                ],
                *[
                    {"role": "assistant", "content": entry["answer"]}
                    for entry in chat_history
                ],
                {"role": "user", "content": question}
            ],
            temperature=0.2,
            max_tokens=300
        )
        
        answer = response["choices"][0]["message"]["content"]
        
        # ✅ 대화 기록 업데이트
        chat_history.append({"question": question, "answer": answer})
        
        return answer

    # 2️⃣ 채용 정보 검색 실행
    courses = fetch_from_saramin_refined()
    documents = convert_to_documents(courses)
    vector_store = create_vector_store(documents)

    # 2️⃣.5️⃣ 유사도 검사 및 응답 생성 (대화 기록 추가)
    answer, source_docs = get_answer_with_similarity_check(question, vector_store, chat_history)

    # 3️⃣ 질문-답변 검증 실행 (유사도 기준 통과한 경우만 검증)
    if source_docs:
        verification_result = verify_answer(question, answer)
        if verification_result["verification"] == "일치":
            return answer  
        else:
        # 검증이 실패했을 경우 사용자에게 다시 질문을 유도하는 메시지 반환
            return "답변이 정확하지 않습니다. 다시 질문해 주세요."
        

    return answer  # 유사도 낮을 경우 질문 유도 메시지 반환

# ✅ 실행 예제
if __name__ == "__main__":
    test_question = "데이터 분석가와 관련된 공고가 궁금해" 
    print(handle_request(test_question))

2025-03-25 02:23:13,497 - INFO - 🟡 [질문 분류] 입력 질문: 데이터 분석가와 관련된 공고가 궁금해
2025-03-25 02:23:13,927 - INFO - 🟢 [질문 분류 완료] 카테고리: 채용 정보
2025-03-25 02:23:13,980 - INFO - 🟢 [DB 데이터 로드 완료] 총 1267개 문서 가져옴
2025-03-25 02:23:14,058 - INFO - 🟢 [문서 변환 완료] 총 2108개 문서 생성됨
2025-03-25 02:23:14,063 - INFO - 🟡 [벡터 스토어 로드] 기존 벡터 스토어를 불러옵니다.
2025-03-25 02:23:14,089 - INFO - 🟢 [벡터 스토어 준비 완료] 문서 개수: 2108
2025-03-25 02:23:15,088 - INFO - 유사도 최고 점수: 0.3259929418563843




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

        시스템 프롬프트:
        1. 당신은 잡Job의 채용 전문가입니다.사용자의 요구에 맞는 채용 정보를 제공하세요.
        2. 채용 공고에 대한 정보를 제공할시 다음과 같은 형식으로 답변하세요. '공고제목', '공고링크', '기업명'을 반드시 포함하세요. 개수와 관련된 명시가 없을시 기본적으로 3개의 공고를 제공하세요.
        - 공고 제목: (공고제목)
        - 공고 링크: (공고링크)
        - 기업명:(기업명)
        - 태그:(태그)
        - 주요 업무: (주요 업무에 대한 설명)
        - 자격 요건: (필수 자격 요건)

        3. 관련 없는 질문에는 자연스럽게 채용 정보 질문을 유도하세요.
        4. 직급과 관련된 답변은 포함하지 마세요. 직급과 관련한 질문에는 공고링크를 제공하세요

        질문: 데이터 분석가와 관련된 공고가 궁금해
        
        관련 문서:
        id: rec-50306090
기업명: (주)트리플하이엠
기업링크: https://www.saramin.co.kr/zf_user/company-info/view-inner-recruit?csn=T1JLcjJQVXVnVzJIK1hrQlBoWmZFZz09
공고제목: [연구소] 온라인 마케팅 데이터 분석 및 설계 (경력)
공고링크: https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50306090
태그: ['데이터라벨링', '데이터마이닝', '데이터시각화', '머신러닝', 'AI(인공지능)']
경력: 
학력: 대졸(2,3년제) 이상
근무형태: 정규직 수습기간 3개월
급여: 면접 후 결정
근무지역: 서울

2025-03-25 02:23:26,391 - INFO - 🟢 [답변 검증 완료] 결과: 일치


다음은 데이터 분석가와 관련된 채용 공고입니다.

1. 
   - 공고 제목: [연구소] 온라인 마케팅 데이터 분석 및 설계 (경력)
   - 공고 링크: [링크](https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=list&rec_idx=50306090)
   - 기업명: (주)트리플하이엠
   - 태그: ['데이터라벨링', '데이터마이닝', '데이터시각화', '머신러닝', 'AI(인공지능)']
   - 주요 업무: 온라인 마케팅 데이터 분석 및 설계, 데이터 분석 제안 기획, Google Analytics 4 구축 및 분석 설계, 고객 Segmentation 기획 및 설계, ML/AI 관련 통계 모델 연구 및 설계
   - 자격 요건: Python, javascript, Bigquery 등 기초적 개발 언어 지식 및 GCP 활용한 데이터 분석 기초 능력 보유, Google Analytics 4 구축 및 분석 경험 보유

2. 
   - 공고 제목: 데이터 분석 및 AI Agent 솔루션 개발
   - 공고 링크: [링크](https://www.example.com)  *(가상의 링크입니다)*
   - 기업명: 비공식 기업명
   - 태그: ['AI', '데이터분석', 'ML', 'DL']
   - 주요 업무: LLM과 GraphRAG 혹은 RAG기술을 활용하여 AI Agent 솔루션 개발, 데이터 EDA 및 가공, 온톨로지 기반 지식그래프 구축 및 그래프알고리즘을 통한 인사이트 도출, ML/DL기술 적용
   - 자격 요건: 경력 1년 ~ 20년, Graph Algorithm 또는 Network Science 기술 경험, SQL 및 Python 능숙, LLM과 Graph RAG 기술에 대한 관심 및 경험

3. 
   - 공고 제목: KPI 설정 및 데이터 분석
   - 공고 링크: [링크](https://www.example.com)  *(가상의 링크입니다)*
   - 기업명: 비공식 기업명
   -